In [2]:
import pandas as pd

ds = pd.read_parquet('./results/ds.parquet')
elapse_time = pd.read_csv('./results/elapse_time.csv')

In [3]:
from bertopic import BERTopic
from gensim.models.ldamulticore import LdaMulticore
from utils import e_variant

def load_ldas():
    r = {} # result
    for v in ds:
        r[v] = LdaMulticore.load(f'./results/models_lda/{v}')
    return r

def load_bertopics():
    path_prefix = './results/models_bertopic/'
    r = {}
    for v in ds:
        r[v] = {}
        r[v] = BERTopic.load(f'{path_prefix}{v}')
    return r

In [4]:
ldas = load_ldas()
bertopics = load_bertopics()

In [5]:
from time import time
from gensim.corpora.dictionary import Dictionary
from utils import get_topics_lda, get_topics_bertopic, get_coherence, get_diversity

def evaluate_models(models, lda=False):
    r = []
    r_time = []
    for v in ds:
        t_start = time()
        docs = ds[v].dropna()
        if lda:
            topics = get_topics_lda(models[v], models[v].id2word)
            texts = docs
            dictionary = models[v].id2word
        else:
            topics = get_topics_bertopic(bertopics[v])
            analyzer = models[v].vectorizer_model.build_analyzer()
            texts = [' '.join(doc) for doc in docs]
            texts = [analyzer(doc) for doc in texts]
            dictionary = Dictionary(texts)
        c = get_coherence(
            topics=topics,
            texts=texts,
            dictionary=dictionary,
        )
        d = get_diversity(topics)
        t_end = time()
        r_time.append(t_end - t_start)
        r.append({
            'variant': v,
            'coherence': c,
            'diversity': d,
            'score': c * d
        })
    algorithm = 'lda' if lda else 'bertopic'
    elapse_time[f'{algorithm}_evaluation'] = pd.Series(r_time)
    return pd.DataFrame(r)

In [6]:
import pandas as pd

evaluation_lda = pd.DataFrame(evaluate_models(ldas, lda=True))

In [7]:
evaluation_bertopic = pd.DataFrame(evaluate_models(bertopics))

In [8]:
elapse_time

,variant,nlp,preprocessing,lda_training,bertopic_training,lda_evaluation,bertopic_evaluation
0,dCL,34.678537,0.311998,34.611548,74.961021,4.770477,3.410751
1,dLWN,34.678537,0.311998,31.916765,15.938241,4.759319,3.980859
2,DCL,34.678537,0.311998,30.490422,37.874624,4.085068,4.437994
3,DLWN,34.678537,0.311998,34.266039,11.792914,4.750199,4.317921


In [9]:
evaluation_lda

,variant,coherence,diversity,score
0,dCL,0.463023,0.137037,0.063451
1,DCL,0.352837,0.137037,0.048352
2,dLWN,0.359462,0.352459,0.126696
3,DLWN,0.396793,0.352459,0.139853


In [10]:
evaluation_bertopic

,variant,coherence,diversity,score
0,dCL,0.405232,1.0,0.405232
1,DCL,0.343653,1.0,0.343653
2,dLWN,0.307534,1.0,0.307534
3,DLWN,0.342356,1.0,0.342356


In [11]:
elapse_time.to_csv('./results/elapse_time.csv', index=False)
evaluation_lda.to_csv('./results/evaluation_lda.csv', index=False)
evaluation_bertopic.to_csv('./results/evaluation_bertopic.csv', index=False)